In [1]:
import pandas as pd
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
# персональные данные необходимые для доступа к информации сайта last.fm
api_key='***'
api_secret='***'

In [4]:
payload = {
    'api_key': api_key,
    'method': 'chart.getTopTracks',
    'format': 'json', 
    'page' : 1
}

r = requests.get('http://ws.audioscrobbler.com/2.0/?method=chart.gettoptracks&api_key=YOUR_API_KEY&format=json', params=payload)

In [5]:
infa=r.json()['tracks']['track']

In [6]:
for i in range(2, 25):
    payload['page']=i
    response=requests.get('http://ws.audioscrobbler.com/2.0/?method=chart.gettoptracks&api_key=YOUR_API_KEY&format=json', params=payload)
    data=response.json()
    infa+=data['tracks']['track']

In [7]:
len(infa) # сколько всего песен получили 

2150

Воспользуемся методом 'chart.getTopTracks' API last.fm и получим песни из чарта вместе с рядом их параметров, мы решили пройтись по 25 страницам, чтобы не усложнять работу имеющихся циклов. 

In [8]:
track_list = []
for track in range(len(infa)):
    track_dict = {'artist_name': infa[track]['artist']['name'], 'song_name': infa[track]['name'],
        'duration': infa[track]['duration'],
        'playcount': infa[track]['playcount'],
        'listeners': infa[track]['listeners']}
    track_list.append(track_dict)
# создали словарь с данными по каждой песне

In [10]:
pip install pylast

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pylast

In [12]:
network = pylast.LastFMNetwork(api_key=api_key, api_secret=api_secret) # установка соединения с сервером

In [13]:
import time
time.sleep(4)

Если посмотреть на infa, то можно заметить, что в песнях присутствует параметр 'duration', однако он везде равен нулю. Поэтому уже для имеющихся песен нам пришлось доставать их длительность и жанр (важные признаки) с помощью уже другого метода track.getInfo, что добавило еще один цикл (но по-другому мы не смогли). 

In [14]:
for track in range(len(track_list)):
    # получаем информацию о треке
    artist_name = track_list[track]['artist_name']
    song_name = track_list[track]['song_name']
    track_info = network.get_track(artist_name, song_name)
    top_tag = track_info.get_top_tags(limit=1)

    track_list[track]['duration'] = track_info.get_duration()
    track_list[track]['tag']=top_tag[0][0].name if top_tag else "Unknown"

In [18]:
df_tracks=pd.DataFrame(track_list)

In [20]:
df_tracks.to_csv('tracks_result.csv', index=False)